# **Spoken Language Understanding** 

## **Introduction**

Spoken Language Understanding (SLU) is a field in NLP that involves elements from speech processing. The field of SLU is targeted at understanding human speech directed at machines (think [Mycroft](https://mycroft.ai/), Siri, Alexa and other voice assistants).

SLU systems typically first parse utterences (or queries) into semantic frames comprised of intents and slots. The intent of a query could be something like "get_direction" for example and the slot could be "cambridge". Architectures have been proposed for filling in intents and slots separately and jointly.(See the references in this [paper](https://www.isca-speech.org/archive/Interspeech_2016/pdfs/1352.PDF) for an overview)

The dataset consists of 44K annotated queries from this [paper](https://arxiv.org/pdf/1810.07942.pdf).

The goal of this task it to semantically parse the user's query to obtain a tree structure with intents and slots as the nodes. For example the query "what's there to do this weekend" should be parsed to [IN:GET_EVENT whats there to do [SL:DATE_TIME this weekend ] ]. Note that the brackets [some span of text] indicate this span is a sub-tree. 
 
In Part B, we will implement a CKY decoding algorithm to decode the final tree based on the classifier we trained in Part A.

Note: The Jupyter notebook framework was created by the excellent TA's for MIT 6.864 Advanced Natural Language Processing, Fall 2020. Pre-processing had been done to enable CKY-style decoding (code [here](https://github.mit.edu/tianxing/mit_6864_hw3_202003)) and the data are all binary trees.

## **Part A**

We formulate the problem as a simple classification problem, the input to the classifier will be (text, span) and the output will be the label of that span.

We apply a model that learns the parsing structures in 4 steps.

1. Enumerate all possible spans of a sentence
2. Generating word and span embeddings
3. Learning span classifications
4. Decoding a tree structure using the classification distributions of spans

### **Setup**

Import libraries

In [0]:
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from span_tree import *

device = "cuda" if torch.cuda.is_available() else "cpu"
assert device == "cuda"   # use gpu!


### **Data Processing**

Load the dataset or corpus, building the dictionary, the label dictionary and the span list.

In [10]:
def tree_dfs(node, span_list, label_dict, mode):
  '''
  We enumerate every node of a tree with depth first search (DFS).
  '''

  # print(node)
  if len(node.children) == 0:
      # print(type(node))
      assert(type(node) == Token)
      cur_span = (node.index, node.index + 1)
      cur_label = label_dict['Token']
      span_list.append([cur_span, cur_label])
      return span_list, label_dict
      
  cur_span = node.get_token_span()
  cur_label = node.label
  
  if node.label in label_dict:
      cur_label = label_dict[node.label]
  elif mode == 'train':
      cur_label = len(label_dict)
      label_dict[node.label] = cur_label
  else:
      cur_label = label_dict['UNK']
  
  span_list.append([cur_span, cur_label])
  
  if len(node.children) > 1: #if only has one child, we will ignore the Token label, otherwise the token span would have two conflicting labels
      for child in node.children:
          span_list, label_dict = tree_dfs(child, span_list, label_dict, mode)
  
  return span_list, label_dict

  
def process_line(line, vocab_dict, label_dict, mode):
  '''
  Processing a line in the corpus.
  line format: Sentence \t Sentence_Tree \n
  
  Example:
      'what is the shortest way home\t
      [IN:GET_DIRECTIONS what [SUB is [SUB the [SUB shortest [SUB way [SL:DESTINATION home ] ] ] ] ] ]\n'
  
  Inputs:
  vocab_dict: vocab dictionary {word: word_index, ...}
  labels_dict: label dictionary {label: label_index, ...}
  '''
  s, s_tree = line.strip().split('\t')
  words = s.split(' ')
  word_ids = []
  
  for word in words:
      if word in vocab_dict:
          word_ids.append(vocab_dict[word])
      elif mode == 'train':
          word_ids.append(len(vocab_dict))
          vocab_dict[word] = len(vocab_dict)
      else:
          word_ids.append(vocab_dict['UNK'])
  
  tree = Tree(s_tree)
  span_list = []
  span_list, label_dict = tree_dfs(tree.root.children[0], span_list, label_dict, mode)
  
  return word_ids, span_list, vocab_dict, label_dict


def process_corpus(corpus_path, mode, vocab_dict=None, label_dict=None):
  '''
  Note that we just adding new words and labels to the dictionaries while building the training set. Unseen words or labels in valid and test set are marked as unknown (UNK).
  '''
  
  lines = open(corpus_path).readlines()
    
  if not vocab_dict:
      vocab_dict = {'UNK': 0}
  if not label_dict:
      label_dict = {'UNK': 0, 'Token': 1, 'None': 2}
  
  corpus = []
  sent_spans = []
  
  for line in lines:
      word_ids, span_list, vocab_dict, label_dict = process_line(line, vocab_dict, label_dict, mode)
      corpus.append(word_ids)
      sent_spans.append(span_list)
  
  return corpus, sent_spans, vocab_dict, label_dict


# corpus_train, spans_train, vocab_dict, label_dict = process_corpus("../data/slu_train.txt", "train") # process_corpus('/content/6864-hw3a/train.txt', 'train')
# corpus_valid, spans_valid, _, _ = process_corpus("../data/slu_valid.txt", "eval", vocab_dict = vocab_dict, label_dict = label_dict)
# corpus_test,  spans_test, _, _  = process_corpus("../data/slu_test.txt", "eval", vocab_dict = vocab_dict, label_dict = label_dict)
corpus_train, spans_train, vocab_dict, label_dict = process_corpus("../data/slu_train_1.txt", "train") # process_corpus('/content/6864-hw3a/train.txt', 'train')

num_words = len(vocab_dict)
num_labels = len(label_dict)

print('Number of different words: {}'.format(num_words))
print('Number of different labels: {}'.format(num_labels))

print(vocab_dict)
print(label_dict)
print()
print(spans_train)

Number of different words: 11
Number of different labels: 8
{'UNK': 0, 'how': 1, 'long': 2, 'will': 3, 'it': 4, 'take': 5, 'to': 6, 'drive': 7, 'from': 8, 'chicago': 9, 'mississippi': 10}
{'UNK': 0, 'Token': 1, 'None': 2, 'IN:GET_ESTIMATED_DURATION': 3, 'SUB': 4, 'SL:METHOD_TRAVEL': 5, 'SL:SOURCE': 6, 'SL:DESTINATION': 7}

[[[(0, 11), 3], [(0, 1), 1], [(1, 11), 4], [(1, 2), 1], [(2, 11), 4], [(2, 3), 1], [(3, 11), 4], [(3, 4), 1], [(4, 11), 4], [(4, 5), 1], [(5, 11), 4], [(5, 6), 1], [(6, 11), 4], [(6, 7), 5], [(7, 11), 4], [(7, 8), 1], [(8, 11), 4], [(8, 9), 6], [(9, 11), 4], [(9, 10), 1], [(10, 11), 7]]]


## Defining the Neural Networks

### Sentence Encoding

We use a Bi-directional LSTM for sentence encoding. We build a sentence encoder with a embedding layer and a Bi-directional LSTM layer.

- Input: word indices [batch_size, sentence_length]
- Output: word embeddings [batch_size, sentence_length, hidden_size]

In [0]:
class SentEnc(nn.Module):
    
    def __init__(self, num_words, num_layers, hidden_size, dropout=0):
        super(SentEnc, self).__init__()
        self.embedding = nn.Embedding(num_words, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first = True, dropout = dropout, bidirectional = True)
    
    def forward(self, x):
        # --------- Your code --------- #
        
        # --------- Your code ends --------- #
        return outputs

### Span Encoding

Given the LSTM outputs, we generate the span embeddings with the span indices.

- Input: word embeddings [sentence_length, hidden_size], span indices [num_span, 2]
- Output: span embeddings [num_span, hidden_size]

We generate a span embedding by concatenate the word embeddings of the first and last words of a span. For example, if a span starts from the i-th word and ends at the j-th word, our span embedding would be

[h_i^T; h_j^T]^T,

where h_i stands for the Bi-LSTM output of the i-th word.

In pytorch, Given the hidden states h[0], h[1], ..., h[n], where
```
h[i].size() = [1, k]
```
the embedding of span (i, j) is
```
span_ij = torch.cat([h[i], h[j]], dim=1)
span_ij.size() = [1, 2 * k]
```
Please complete the following function for generating span embeddings.

Inputs:
- word_embeddings (LSTM outputs) [n_words, hidden_size]
- Span indices [n_spans, 2]

Outputs:
- span embeddings [n_spans, hidden_size * 2]


In [0]:
def get_span_embeddings(word_embeddings, span_indices):
    # --------- Your code --------- #
    
    # --------- Your code ends --------- #

### Tag Prediction

We build a Classifier that puts the neural models together. The classifier takes word and span indices as inputs, and predict span labels by calculating word embeddings, span embeddings, and label logits. we will predict the tag of the spans with a linear classifier.

- Inputs: word indices: [batch_size, num_words]
- Outputs: span predictions: [num_spans, num_labels]

Please implement the forward function following the 3 steps:
1. Generate the word embeddings by processing the input sentences with the LSTM sentence encoder.
2. Apply dropout on word embeddings.
3. Calculate span embeddings with function get_span_embeddings().
4. Calculate label logits with the linear layer defined as follow.

In [0]:
class Classifier(nn.Module):
    
    def __init__(self, num_words, num_labels, num_layers, hidden_size, dropout=0):
        super(Classifier, self).__init__()
        self.sent_enc = SentEnc(num_words, num_layers, hidden_size)
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(4 * hidden_size, num_labels)
    
    def forward(self, x, span_indices):
        # --------- Your code --------- #
        
        # --------- Your code ends --------- #
        return logits

## Training Loop

With all neural models already defined, we are implementing the training loop.

In [0]:
#For decoding, we add some random spans and label them as "None"
def add_none_span(word_list, span_list, label_dict, all=False):
    num_words = len(word_list)
    num_labeled_span = len(span_list)
    labeled_span_set = set([span for span, label in span_list])
    none_spans = []
    for i in range(num_words):
        for j in range(i + 1, num_words):
            if (i, j) not in labeled_span_set:
                none_spans.append([(i, j), label_dict['None']])
    if not all:
        k = min(num_labeled_span, len(none_spans))
        sampled_none_spans = random.sample(none_spans, k)
    else:
        sampled_none_spans = none_spans
    return span_list + sampled_none_spans

print('Using device: {}'.format(device))

# --------- Your code --------- #
# There's no code here, just remeber you can tune these hyper-parameters!
# --------- Your code ends --------- #
batch_size = 1
num_layers = 2
hidden_size = 200
lr = 0.05
num_epochs = 3 #Be aware of over-fitting!
loss_fn = nn.CrossEntropyLoss().to(device)
dropout = 0.25

classifier = Classifier(num_words, num_labels, num_layers, hidden_size, dropout)
optimizer = optim.SGD(classifier.parameters(), lr=lr, momentum=0.9)

classifier = classifier.to(device)
classifier.train()

for epoch in range(num_epochs):
    total_loss = 0
    classifier.train()
    for i in range(len(corpus_train)):

        if i % 10000 == 0:
            print('Epoch {} Batch {}'.format(epoch, i))
        
        cur_spans = add_none_span(corpus_train[i], spans_train[i], label_dict)
        
        sent_inputs  = torch.Tensor([corpus_train[i]]).long().to(device)
        span_indices = torch.Tensor([x[0] for x in cur_spans]).long().to(device)
        span_labels  = torch.Tensor([x[1] for x in cur_spans]).long().to(device)
        
        # --------- Your code --------- #

        # --------- Your code ends --------- #
    print('Epoch {}, train loss={}'.format(epoch, total_loss / len(corpus_train)))

    total_loss = 0
    classifier.eval()
    for i in range(len(corpus_valid)):
        #if i % 10000 == 0:
        #    print('Epoch {} Batch {}'.format(epoch, i))
        cur_spans = add_none_span(corpus_valid[i], spans_valid[i], label_dict)
        
        sent_inputs  = torch.Tensor([corpus_valid[i]]).long().to(device)
        span_indices = torch.Tensor([x[0] for x in cur_spans]).long().to(device)
        span_labels  = torch.Tensor([x[1] for x in cur_spans]).long().to(device)
        
        # --------- Your code --------- #

        # --------- Your code ends --------- #
    print('Epoch {}, valid loss={}'.format(epoch, total_loss / len(corpus_valid)))

## Evaluation

After training the model, we evaluate the classification result.  
What we will do is that we treat a tree strcture as a bag of spans, and then compute F-1 score.  
You don't need to write any code in this section. But it's good for you to understand what it's doing.

In [0]:
from itertools import zip_longest
from typing import Counter, Dict, Optional
import numpy as np

class Calculator:
    def __init__(self, strict: bool = False) -> None:
        self.num_gold_nt: int = 0
        self.num_pred_nt: int = 0
        self.num_matching_nt: int = 0
        self.strict: bool = strict
        self.exact_match = []
        self.well_form = []

    def get_metrics(self):
        precision: float = (
            self.num_matching_nt / self.num_pred_nt) if self.num_pred_nt else 0
        recall: float = (
            self.num_matching_nt / self.num_gold_nt) if self.num_gold_nt else 0
        f1: float = (2.0 * precision * recall /
                     (precision + recall)) if precision + recall else 0
        
        return {
            "precision": precision,
            "recall": recall,
            "f1": f1,
            "exact_match": np.mean(self.exact_match),
            "well_form": np.mean(self.well_form),
        }
    
    def add_instance_span(self, gold_spans, pred_spans):
        self.num_gold_nt += len(gold_spans)
        self.num_pred_nt += len(pred_spans)
        gold_spans = set(gold_spans)
        pred_spans = set(pred_spans)
        self.num_matching_nt += len(gold_spans & pred_spans)
        self.exact_match.append(int(gold_spans == pred_spans))
        well_form = True
        for s1 in pred_spans: 
            for s2 in pred_spans:
                if s1[0] < s2[0] and s2[0] < s1[1] and s1[1] < s2[1]:
                    well_form = False
        self.well_form.append(int(well_form))

    def add_instance_tree(self, gold_tree: Tree,
                     pred_tree: Optional[Tree] = None) -> None:
        node_info_gold: Counter = self._get_node_info(gold_tree)
        self.num_gold_nt += sum(node_info_gold.values())
            
        if pred_tree:
            node_info_pred: Counter = self._get_node_info(pred_tree)
            self.num_pred_nt += sum(node_info_pred.values())
            self.num_matching_nt += sum(
                (node_info_gold & node_info_pred).values())
            self.exact_match.append(int(node_info_gold.keys() == node_info_pred.keys()))
            self.well_form.append(1) #we assume the decoded tree is indeed a tree :)
        
    def _get_node_info(self, tree) -> Counter:
        nodes = tree.root.list_nonterminals()
        node_info: Counter = Counter()
        for node in nodes:
            #node_info[(node.label, self._get_span(node))] += 1 #here I change it to only care about the span
            node_info[(self._get_span(node))] += 1
        return node_info

    def _get_span(self, node):
        return node.get_flat_str_spans(
        ) if self.strict else node.get_token_span()


classifier.eval()
num_correct = 0.
num_spans = 0
calc = Calculator(strict=False)

for kk in range(len(corpus_test)):
    #we will create bag of spans based on our prediction

    #cur_spans = add_none_span(corpus_test[i], spans_test[i], label_dict)
    cur_spans = spans_test[kk]
    cur_spans = [x for x in cur_spans if x[0][1] > x[0][0] + 1] #only test non-terminals
    #print(cur_spans)
    if len(cur_spans) <= 1: continue
    gold_spans = [x[0] for x in cur_spans]

    tokens = corpus_test[kk]
    cur_len = len(corpus_test[kk])
    sent_inputs  = torch.Tensor([tokens]).long().to(device)
    all_spans = []
    for i in range(cur_len):
        for j in range(i + 2, cur_len + 1):
            all_spans.append((i,j))
    logits = classifier(sent_inputs, torch.Tensor(all_spans).long().to(device))
    #logprobs = torch.log(torch.softmax(logits, dim = -1))
    
    pred_spans = []
    for i, span in enumerate(all_spans):
        if torch.argmax(logits[i]).item() != 2:
            pred_spans.append(span)
    
    calc.add_instance_span(gold_spans, pred_spans)
 
print(calc.get_metrics())


## **Part B**  

In part B, we will decode a tree based on the classifier trained on part A.  
Important hint: You should avoid the "None" type during the decoding.  

You will be implementing the following simple CYK recursion:  
best_score[i, j] = max_k {best_score[i, k] + best_score[k, j]} + max_l {span_dict[(i, j)][l]}  
where l is the label of the current span (i,j), and k is the splitting point

In [0]:
dp_results = []
classifier.eval()
for kk, line in enumerate(open('/content/6864-hw3a/test.txt', 'r').readlines()):
    line = line.strip()
    if len(line) < 3: continue
    if kk < 10:
        print(kk, line)
    tokens = corpus_test[kk]
    sent_inputs  = torch.Tensor([corpus_test[kk]]).long().to(device)
    all_spans = []
    for i in range(len(tokens)):
        for j in range(i + 1, len(tokens) + 1):
            all_spans.append((i,j))
    logits = classifier(sent_inputs, torch.Tensor(all_spans).long().to(device))
    logprobs = torch.log(torch.softmax(logits, dim = -1))
    span_dict = {}
    for i, s in enumerate(all_spans): span_dict[s] = logprobs[i] #span dict will map each span (l,r) to its predicted distribution of labels
    TOKEN_ID, NULL_ID = 1, 2
    best_score, best_split, best_label = {}, {}, {} #we will do dynamic programming to decode a binary tree out of our predictions
    
    #Think: why do we first iterate the length of the span?
    for ll in range(1, len(tokens) + 1): #length of the span
        for i in range(0, len(tokens)): #start of the span
            if i + ll > len(tokens): break
            j = i + ll; cur_span = (i, j)
            if j == i + 1:
                # --------- Your code --------- #
                #use span_dict[cur_span] to update best_label and best_score, be careful, it could either be a TOKEN or something else

                # --------- Your code ends --------- #
            else:
                span_dict[cur_span][NULL_ID] = -100000 #we will never decode a NULL sub-tree
                span_dict[cur_span][TOKEN_ID] = -100000 #we will never decode a NULL sub-tree
                # --------- Your code --------- #
                #try to give the values for best_score/label/split[cur_span]

                # --------- Your code ends --------- #
            #print(cur_span, best_score[cur_span], best_label[cur_span])
    dp_results.append((best_score, best_split, best_label))
print(len(dp_results))

In the next section, we will construct a tree using the dp results.  
Before start doing it, please get yourself a little familiar with the span_tree.py.

In [0]:
inv_label_dict = {}
for l in label_dict: inv_label_dict[label_dict[l]] = l
#print(inv_label_dict)

def get_nodetype(label):
    if label.startswith(PREFIX_INTENT):
        node = Intent(label)
    elif label.startswith(PREFIX_SLOT):
        node = Slot(label)
    elif label.startswith(PREFIX_SUBTREE):
        node = SubTree(label)
    else:
        print('something wrong with the label!!!', label, l, r)
        sys.exit(1)
    return node

def dfs_build(l, r):

    if l + 1 == r:
        la = best_label[(l,r)]
        if la == 1:
            return Token(surface_tokens[l], l)
        else:
            node = get_nodetype(inv_label_dict[la])
            node.children = [Token(surface_tokens[l], l)]
            node.children[0].parent = node
            return node

    label = inv_label_dict[best_label[(l, r)]]
    node = get_nodetype(label)
    
    #--- your code --- #
    #hint: only need one line, use best_split!

    #--- your code ends --- #

    for c in node.children:
        c.parent = node
    
    return node

pred_trees = []
for kk, line in enumerate(open('/content/6864-hw3a/test.txt', 'r').readlines()):
    tt = line.strip().split('\t')
    surface_tokens, str_ref_tree = tt[0].split(), tt[1]
    if len(line) < 3: continue
    tokens = corpus_test[kk]
    cur_len = len(tokens)
    best_score, best_split, best_label = dp_results[kk]
    #print(inv_label_dict[best_label[(0, cur_len)]], best_split[(0, cur_len)])
    root = Root()
    root.children = [dfs_build(0, cur_len)]
    root.children[0].parent = root
    tree = Tree('IN:GET_EVENT placeholder') #the string here is just a placeholder
    tree.root = root
    if kk < 10: #use this info for debugging! Does your tree make sense?
        print(kk, line.strip())
        print('REF:', str_ref_tree)
        print('DEC:', str(tree))
        print()
    """ here's some decoding examples we get
    0 whats there to do this weekend	[IN:GET_EVENT whats [SUB there [SUB to [SUB do [SL:DATE_TIME this weekend ] ] ] ] ]
    REF: [IN:GET_EVENT whats [SUB there [SUB to [SUB do [SL:DATE_TIME this weekend ] ] ] ] ]
    DEC: [IN:GET_EVENT whats [SUB there [SUB to [SUB do [SL:DATE_TIME this weekend ] ] ] ] ]

    1 what is a good restaurant for tex mex in austin	[IN:UNSUPPORTED what [SUB is [SUB a [SUB good [SUB restaurant [SUB for [SUB tex [SUB mex [SUB in austin ] ] ] ] ] ] ] ] ]
    REF: [IN:UNSUPPORTED what [SUB is [SUB a [SUB good [SUB restaurant [SUB for [SUB tex [SUB mex [SUB in austin ] ] ] ] ] ] ] ] ]
    DEC: [IN:UNSUPPORTED what [SUB is [SUB a [SUB good [SUB restaurant [SUB for [SUB tex [SUB mex [SUB in austin ] ] ] ] ] ] ] ] ]

    2 where can i see the fireworks tonight	[IN:GET_EVENT where [SUB can [SUB i [SUB see [SUB [SL:CATEGORY_EVENT the fireworks ] [SL:DATE_TIME tonight ] ] ] ] ] ]
    REF: [IN:GET_EVENT where [SUB can [SUB i [SUB see [SUB [SL:CATEGORY_EVENT the fireworks ] [SL:DATE_TIME tonight ] ] ] ] ] ]
    DEC: [IN:GET_EVENT where [SUB can [SUB i [SUB see [SUB the [SUB fireworks [SL:DATE_TIME tonight ] ] ] ] ] ] ]
    """
    pred_trees.append(tree)

Finally, we will compute the F1 score, using the pred_trees.  
You don't need to write any code in this section.

In [0]:
print(len(pred_trees))
#compute the score!
labeled_bracketing_scores = Calculator(strict=False)

for kk, line in enumerate(open('/content/6864-hw3a/test.txt', 'r').readlines()):
    tt = line.strip().split('\t')
    surface_tokens, str_ref_tree = tt[0].split(), tt[1]
    if len(line) < 3: continue
    tokens = corpus_test[kk]
    labeled_bracketing_scores.add_instance_tree(Tree(str_ref_tree), pred_trees[kk])

print(labeled_bracketing_scores.get_metrics())